In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install torchmetrics

In [4]:
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.3 MB/s 
     |████████████████████████████████| 131 kB 22.9 MB/s 
  Created wheel for kss: filename=kss-3.5.6-py3-none-any.whl size=42448525 sha256=7d1b74184dfd3e843acc2007bb0d565e4a4a6cd8d7df5eef80cdec28cc79902c
  Stored in directory: /root/.cache/pip/wheels/f7/1e/7d/a31add1236183845fb9b2dfadd50636429a5fbcabd5f341c86
Successfully built kss


In [5]:
import math
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from transformers import BertModel, BertTokenizer, AdamW, PreTrainedTokenizerFast, BartModel

from torch.nn.init import xavier_uniform_

import pytorch_lightning as pl

import kss
import re

In [6]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


# class

In [7]:
class SummDataset(Dataset):

    def __init__(
        self, 
        data: pd.DataFrame, 
        tokenizer: PreTrainedTokenizerFast, 
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(tokenizer(
                text = sent,
                add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'
    
        src = [] # 토크나이징 된 전체 문단
        labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
        segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
        clss = []  #[CLS]토큰의 포지션값을 지정

        odd = 0
        for tkns in tokenlist:
            if odd > 1 : odd = 0
            clss = clss + [len(src)]
            src = src + tkns['input_ids']
            segs = segs + [odd] * len(tkns['input_ids'])
            if tokenlist.index(tkns) in data_row.extractive :
                labels = labels + [1]
            else:
                labels = labels + [0]
            odd += 1
        
            #truncation
            if len(src) == MAX_TOKEN_COUNT:
                break
            elif len(src) > MAX_TOKEN_COUNT:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break
    
        #padding
        if len(src) < MAX_TOKEN_COUNT:
            src = src + [0]*(self.max_token_len - len(src))
            segs = segs + [0]*(self.max_token_len - len(segs))
            
        if len(clss) < MAX_TOKEN_COUNT:
            clss = clss + [-1]*(self.max_token_len - len(clss))
        if len(labels) < MAX_TOKEN_COUNT:
            labels = labels + [0]*(self.max_token_len - len(labels))

        return dict(
            src = torch.tensor(src),
            segs = torch.tensor(segs),
            clss = torch.tensor(clss),
            labels= torch.FloatTensor(labels)
        )

In [8]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

In [9]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6) 
        self.dropout = nn.Dropout(dropout) 

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

In [10]:
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.3, num_inter_layers=2): 
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask) 

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores

In [11]:
class PositionwiseFeedForward(nn.Module):
    """ A two-layer Feed-Forward-Network with residual layer norm.

    Args:
        d_model (int): the size of input for the first-layer of the FFN.
        d_ff (int): the hidden layer size of the second-layer
            of the FNN.
        dropout (float): dropout probability in :math:`[0, 1)`.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x

In [12]:
class MultiHeadedAttention(nn.Module):
    """
    Args:
       head_count (int): number of parallel heads
       model_dim (int): the dimension of keys/values/queries,
           must be divisible by head_count
       dropout (float): dropout parameter
    """

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)
    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)
        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                     self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                     layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                 self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask, -1e18) # how can i fix it to use fp16...

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context

In [13]:
class Summarizer(pl.LightningModule): #

    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained('gogamza/kobart-base-v1') #, return_dict=True)
        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')
    
        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None): #, input_ids, attention_mask, labels=None):
        
        mask_src = ~(src == 0) #1 - (src == 0)
        mask_cls = ~(clss == -1) #1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state
        
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        
        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)
            
            loss = (loss * mask_cls.float()).sum() / len(labels)
        
        return loss, sent_scores

    def step(self, batch):

        src = batch['src']
        if len(batch['labels']) > 0 :
            labels = batch['labels']
        else:
            labels = None
        segs = batch['segs']
        clss = batch['clss']
        
        loss, sent_scores = self(src, segs, clss, labels)    
        
        return loss, sent_scores, labels

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 저장된 모델 불러오기
trained_model = torch.load('/content/drive/MyDrive/데이터/금융 요약/model_fulldata2.pt') 

# 요약하기

In [ ]:
# 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/데이터/금융 요약/2022-08-23(1).csv")
df.head()

,url,title,text
0,https://finance.naver.com/news/news_read.naver...,잭슨 홀 리스크·유럽 침체우려에도 나스닥 소폭 상승 출발 [데일리 국제금융시장],제롬 파월 미 연방준비제도(Fed·연준) 의장이 잭슨 홀 미팅 연설에서 매파적인 모...
1,https://finance.naver.com/news/news_read.naver...,[Asia마감] 긴축 공포에 짓눌린 시장…닛케이 1.19%↓,"23일 아시아 주요국 증시가 일제히 하락 마감했다. 통화긴축, 경기침체에 대한 우려..."
2,https://finance.naver.com/news/news_read.naver...,[마켓뷰] 코스피·코스닥 1% 넘게 하락…치솟는 美 달러화 가치에 투자금 ‘썰물’,23일 코스피지수가 1% 넘게 하락했다. 미 달러화 가치가 초강세를 지속하고 있고...
3,https://finance.naver.com/news/news_read.naver...,"[마감시황]코스피, 강달러 지속에 1%대 하락…2430선 마감",[서울=뉴시스] 코스닥지수는 전 거래일(795.87)보다 12.45포인트(1.56%...
4,https://finance.naver.com/news/news_read.naver...,[유럽개장]장 초반 하락세…獨 0.35%↓,23일 유럽 증시가 장 초반 하락세를 보이고 있다.이날 오후 4시20분(한국시간) ...


In [ ]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 3
BATCH_SIZE = 2

In [ ]:
def data_process(text):
    # 문장 분리 하고,
    text = re.sub('\[.*\]|\s-\s.*', '', text)
    sents = kss.split_sentences(text)
    
    #데이터 가공하고,
    tokenlist = []
    for sent in sents:
        tokenlist.append(tokenizer(
            text = sent,
            add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'

    src = [] # 토크나이징 된 전체 문단
    labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
    segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
    clss = []  #[CLS]토큰의 포지션값을 지정

    odd = 0

    for tkns in tokenlist:

        if odd > 1 : odd = 0
        clss = clss + [len(src)]
        src = src + tkns['input_ids']
        segs = segs + [odd] * len(tkns['input_ids'])
        odd += 1

        #truncation
        if len(src) == MAX_TOKEN_COUNT:
            break
        elif len(src) > MAX_TOKEN_COUNT:
            src = src[:MAX_TOKEN_COUNT - 1] + [src[-1]]
            segs = segs[:MAX_TOKEN_COUNT]
            break

    #padding
    if len(src) < MAX_TOKEN_COUNT:
        src = src + [0]*(MAX_TOKEN_COUNT - len(src))
        segs = segs + [0]*(MAX_TOKEN_COUNT - len(segs))

    if len(clss) < MAX_TOKEN_COUNT:
        clss = clss + [-1]*(MAX_TOKEN_COUNT - len(clss))

    return dict(
        sents = sents, #정답 출력을 위해...
        src = torch.tensor(src),
        segs = torch.tensor(segs),
        clss = torch.tensor(clss),
    )

In [ ]:
def summarize_test(text):
    data = data_process(text.replace('\n',''))
    
    #trained_model에 넣어 결과값 반환
    _, rtn = trained_model(data['src'].unsqueeze(0), data['segs'].unsqueeze(0), data['clss'].unsqueeze(0))
    rtn = rtn.squeeze()
    
    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending = True)
    
    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    end_idx = rtn_sort.index(0)

    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]
    
    if len(idx) > 3:
        rslt = idx[:3]
    else:
        rslt = idx
        
    summ = []
    print(' ')
    for i, r in enumerate(rslt):
        summ.append(data['sents'][r])
        #print(summ[i])

    return summ

In [ ]:
sentence = []
for news_sentence in df['text'].values:
    sentence.append(news_sentence)

In [ ]:
array = []
for i in sentence:
  sum_list = summarize_test(i)
  sum = " ".join(sum_list)
  array.append(sum)
print(array)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
['제롬 파월 미 연방준비제도(Fed·연준) 의장이 잭슨 홀 미팅 연설에서 매파적인 모습을 보일 수 있다는 우려가 커지는 가운데, 유럽의 경기침체 가능성이 크게 높아지면서 미국 뉴욕증시의 주요 지수가 출발했다. 23일(현지 시간) 미 경제 방송 CNBC에 따르면 이날 오전9시35분 현재 뉴욕증권거래소(NYSE)의 다우존스산업평균지수가 전날보다 21.58포인트(0.065%) 오른 3만3085.19에 거래되고 있다. 스탠더드앤드푸어스(S&P)500이 2.87포인트(0.069%) 상승한 4140.86, 나스닥은 23.77포인트(0.18%) 오른 1만2405.34를 기록 중이다.', '나흘 연속 하락한 것이다. 이날 일본 도쿄 주식시장에서 닛케이225지수는 전날 대비 341.75포인트(1.19%) 하락한 2만8452.75로 장을 마감했다. 23일 아시아 주요국 증시가 일제히 하락 마감했다.', '23일 오후 서울 중구 하나은행 딜링룸 전광판에 코스피지수, 미 달러화에 대한 원화 환율이 표시돼 있다. 이날 코스피지수는 전 거래일보다 27.16포인트(1.1%) 내린 2435.34로 마감했다. 미 달러화 가치가 초강세를 지속하고 있고, 유럽의 경기 침체에 대한 우려가 커지자, 우리 증시도 타격을 입은 것으로 해석된다.', '23일 코스피는 전 거래일 대비 27.16포인트(1.10%) 내린 2435.34에 거래를 마쳤다. 코스피가 달러 강세가 지속된 영향에 1%대 하락 마감했다. 원·달러 환율은 전 거래일(1339.8원)보다 5.7원 오른 1345.5원에 마감했다.', '영국 FTSE지수와 프랑스CAC지수도 전 거래일 대비 각각 0.31%, 0.35% 하락한 7510.13, 6356.67를 기록 중이다. 이날 오후 4시20분(한국시간) 현재 독일 DAX지수는 전 거래일보다 0.35% 하락한 1만3184.47으로 거래되고 있다. 23일 유럽 증시가 장 초반 하락세를 보이고 있다.', '삼성전자, LG화학 등 코스피 시가총액 상

In [ ]:
df['추출요약']=array
df

,url,title,text,추출요약
0,https://finance.naver.com/news/news_read.naver...,잭슨 홀 리스크·유럽 침체우려에도 나스닥 소폭 상승 출발 [데일리 국제금융시장],제롬 파월 미 연방준비제도(Fed·연준) 의장이 잭슨 홀 미팅 연설에서 매파적인 모...,제롬 파월 미 연방준비제도(Fed·연준) 의장이 잭슨 홀 미팅 연설에서 매파적인 모...
1,https://finance.naver.com/news/news_read.naver...,[Asia마감] 긴축 공포에 짓눌린 시장…닛케이 1.19%↓,"23일 아시아 주요국 증시가 일제히 하락 마감했다. 통화긴축, 경기침체에 대한 우려...",나흘 연속 하락한 것이다. 이날 일본 도쿄 주식시장에서 닛케이225지수는 전날 대비...
2,https://finance.naver.com/news/news_read.naver...,[마켓뷰] 코스피·코스닥 1% 넘게 하락…치솟는 美 달러화 가치에 투자금 ‘썰물’,23일 코스피지수가 1% 넘게 하락했다. 미 달러화 가치가 초강세를 지속하고 있고...,"23일 오후 서울 중구 하나은행 딜링룸 전광판에 코스피지수, 미 달러화에 대한 원화..."
3,https://finance.naver.com/news/news_read.naver...,"[마감시황]코스피, 강달러 지속에 1%대 하락…2430선 마감",[서울=뉴시스] 코스닥지수는 전 거래일(795.87)보다 12.45포인트(1.56%...,23일 코스피는 전 거래일 대비 27.16포인트(1.10%) 내린 2435.34에 ...
4,https://finance.naver.com/news/news_read.naver...,[유럽개장]장 초반 하락세…獨 0.35%↓,23일 유럽 증시가 장 초반 하락세를 보이고 있다.이날 오후 4시20분(한국시간) ...,"영국 FTSE지수와 프랑스CAC지수도 전 거래일 대비 각각 0.31%, 0.35% ..."
5,https://finance.naver.com/news/news_read.naver...,"[시황종합] 코스피, 강달러 충격에 5일째 하락 2435선 마감…환율 1345.5원",23일 코스피지수가 달러 강세의 영향으로 1%가량 하락하며 2435선에서 마감했다....,"삼성전자, LG화학 등 코스피 시가총액 상위종목뿐만 아니라 셀트리온헬스케어, 에코프..."
6,https://finance.naver.com/news/news_read.naver...,"[외환마감]환율, 1340원대 연고점 경신…尹대통령 언급+당국개입도 역부족",원·달러 환율이 외환당국의 직접 개입에도 불구하고 6원 가까이 오르며 1340원대로...,"연고점은 물론, 금융위기 이후 13년 4개월만에 최고치를 이틀 연속 경신한 것으로 ..."
7,https://finance.naver.com/news/news_read.naver...,[코스닥 마감]킹달러·잭슨홀 경계감에 1.5%↓…780선 턱걸이,코스닥 지수가 주요국 중앙은행 총재들이 모이는 잭슨홀 미팅을 앞두고 경계감이 작동하...,코스닥 지수가 주요국 중앙은행 총재들이 모이는 잭슨홀 미팅을 앞두고 경계감이 작동하...
8,https://finance.naver.com/news/news_read.naver...,[Asia오전] 美 추락에 亞증시도 '뚝뚝뚝'…닛케이 1.17%↓,"22일 오전 아시아 증시는 모두 하락세를 나타냈다. 통화긴축, 경기침체 대한 우려가...",일본 도쿄증시의 닛케이225지수는 전일 대비 337.58포인트(1.17%) 추락한 ...
9,https://finance.naver.com/news/news_read.naver...,"[외환브리핑]109선 찍은 달러인덱스…환율, 1340원대 안착 시도",원·달러 환율이 6거래일째 올라 1340원대 상승을 시도할 전망이다. 전날 13.9...,전날 13.9원이나 올랐으나 1340원대 안착에는 실패했지만 이날은 우리시간으로 열...
